d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Corrupt Record Handling

Apache Spark&trade; and Databricks&reg; provide ways to handle corrupt records.

## In this lesson you:
* Define corruption logic to handle corrupt records
* Pipe corrupt records into a directory for later analysis

## Audience
* Primary Audience: Data Engineers
* Additional Audiences: Data Scientists and Software Engineers

## Prerequisites
* Web browser: Please use a <a href="https://docs.databricks.com/user-guide/supported-browsers.html#supported-browsers" target="_blank">supported browser</a>.
* Concept (optional): <a href="https://academy.databricks.com/collections/frontpage/products/dataframes" target="_blank">DataFrames course from Databricks Academy</a>

<iframe  
src="//fast.wistia.net/embed/iframe/4m6z26iu8h?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/4m6z26iu8h?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

-sandbox
## Working with Corrupt Data

ETL pipelines need robust solutions to handle corrupt data. This is because data corruption scales as the size of data and complexity of the data application grow. Corrupt data includes:  
<br>
* Missing information
* Incomplete information
* Schema mismatch
* Differing formats or data types
* User errors when writing data producers

Since ETL pipelines are built to be automated, production-oriented solutions must ensure pipelines behave as expected. This means that **data engineers must both expect and systematically handle corrupt records.**

In the road map for ETL, this is the **Handle Corrupt Records** step:
<img src="https://files.training.databricks.com/images/eLearning/ETL-Part-1/ETL-Process-3.png" style="border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa"/>

<iframe  
src="//fast.wistia.net/embed/iframe/5y70n1k6vz?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/5y70n1k6vz?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Setup & Classroom-Cleanup<br>

For each lesson to execute correctly, please make sure to run the **`Classroom-Setup`** cell at the start of each lesson (see the next cell) and the **`Classroom-Cleanup`** cell at the end of each lesson.

In [0]:
%run "./Includes/Classroom-Setup"

Preparing learning environment...

Configuing courseware for Python...

Defining courseware-specific utility methods...

Defining lesson-specific utility methods...
 
 untilStreamIsReady(name) : Utility method to block execution until the stream is fully initialized. 
 stopAllStreams() : Utility method to stop all active streams in a fail-safe manner.

Initializing test framework.

...

Running on DBR 6.3 (6.3.x-scala2.11)

Initializing services...

Defined the following values:
 
 moduleName : etl-part-1 
 lessonName : ETL1 06 - Corrupt Record Handling 
 username : vivek.sivalingam@rhsmith.umd.edu 
 userhome : dbfs:/user/vivek.sivalingam@rhsmith.umd.edu 
 workingDir : dbfs:/user/vivek.sivalingam@rhsmith.umd.edu/etl_part_1/etl1_06___corrupt_record_handling_psp 
 

Created a lesson-specific database:
 
 databaseName : vivek_sivalingam_rhsmith_umd_edu_etl_part_1_etl1_06___corrupt_record_handling_dbpsp

Removing: dbfs:/user/vivek.sivalingam@rhsmith.umd.edu/etl_part_1/etl1_06___corrupt_record_handling_psp/corruptSMS/
Removing: dbfs:/user/vivek.sivalingam@rhsmith.umd.edu/etl_part_1/etl1_06___corrupt_record_handling_psp

All done!

-sandbox
Run the following cell, which contains a corrupt record, `{"a": 1, "b, "c":10}`:

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> This is not the preferred way to make a DataFrame.  This code allows us to mimic a corrupt record you might see in production.

In [0]:
data = """{"a": 1, "b":2, "c":3}|{"a": 1, "b":2, "c":3}|{"a": 1, "b, "c":10}""".split('|')

corruptDF = (spark.read
  .option("mode", "PERMISSIVE")
  .option("columnNameOfCorruptRecord", "_corrupt_record")
  .json(sc.parallelize(data))
)

display(corruptDF)

_corrupt_record,a,b,c
null,1,2,3
null,1,2,3
"{""a"": 1, ""b, ""c"":10}",null,null,null


In the previous results, Spark parsed the corrupt record into its own column and processed the other records as expected. This is the default behavior for corrupt records, so you didn't technically need to use the two options `mode` and `columnNameOfCorruptRecord`.

There are three different options for handling corrupt records [set through the `ParseMode` option](https://github.com/apache/spark/blob/master/sql/catalyst/src/main/scala/org/apache/spark/sql/catalyst/util/ParseMode.scala#L34):

| `ParseMode` | Behavior |
|-------------|----------|
| `PERMISSIVE` | Includes corrupt records in a "_corrupt_record" column (by default) |
| `DROPMALFORMED` | Ignores all corrupted records |
| `FAILFAST` | Throws an exception when it meets corrupted records |

The following cell acts on the same data but drops corrupt records:

In [0]:
data = """{"a": 1, "b":2, "c":3}|{"a": 1, "b":2, "c":3}|{"a": 1, "b, "c":10}""".split('|')

corruptDF = (spark.read
  .option("mode", "DROPMALFORMED")
  .json(sc.parallelize(data))
)
display(corruptDF)

a,b,c
1,2,3
1,2,3


The following cell throws an error once a corrupt record is found, rather than ignoring or saving the corrupt records:

In [0]:
try:
  data = """{"a": 1, "b":2, "c":3}|{"a": 1, "b":2, "c":3}|{"a": 1, "b, "c":10}""".split('|')

  corruptDF = (spark.read
    .option("mode", "FAILFAST")
    .json(sc.parallelize(data))
  )
  display(corruptDF)
  
except Exception as e:
  print(e)

An error occurred while calling o801.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 178.0 failed 1 times, most recent failure: Lost task 7.0 in stage 178.0 (TID 4446, localhost, executor driver): org.apache.spark.SparkException: Malformed records are detected in schema inference. Parse Mode: FAILFAST.
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1.apply(JsonInferSchema.scala:66)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1.apply(JsonInferSchema.scala:53)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:435)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:441)
	at scala.collection.Iterator$class.isEmpty(Iterator.scala:331)
	at scala.collection.AbstractIterator.isEmpty(Iterator.scala:1334)
	at scala.collection.TraversableOnce$class.reduceLeftOption(TraversableOnce.scala:203)
	at scala.collection.AbstractIterator.reduceLeftOption(Iterator.scala:1334)
	at scala.collection.TraversableOnce$class.reduceOption(TraversableOnce.scala:210)
	at scala.collection.AbstractIterator.reduceOption(Iterator.scala:1334)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1.apply(JsonInferSchema.scala:70)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1.apply(JsonInferSchema.scala:50)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:830)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:830)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:60)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:353)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:317)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:140)
	at org.apache.spark.scheduler.Task.run(Task.scala:113)
	at org.apache.spark.executor.Executor$TaskRunner$$anonfun$13.apply(Executor.scala:533)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1541)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:539)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: com.fasterxml.jackson.core.JsonParseException: Unexpected character ('c' (code 99)): was expecting a colon to separate field name and value
 at [Source: [B@512818f1; line: 1, column: 16]
	at com.fasterxml.jackson.core.JsonParser._constructError(JsonParser.java:1581)
	at com.fasterxml.jackson.core.base.ParserMinimalBase._reportError(ParserMinimalBase.java:533)
	at com.fasterxml.jackson.core.base.ParserMinimalBase._reportUnexpectedChar(ParserMinimalBase.java:462)
	at com.fasterxml.jackson.core.json.UTF8StreamJsonParser._skipColon2(UTF8StreamJsonParser.java:3009)
	at com.fasterxml.jackson.core.json.UTF8StreamJsonParser._skipColon(UTF8StreamJsonParser.java:2984)
	at com.fasterxml.jackson.core.json.UTF8StreamJsonParser.nextToken(UTF8StreamJsonParser.java:744)
	at org.apache.spark.sql.catalyst.json.JacksonUtils$.nextUntil(JacksonUtils.scala:29)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$.inferField(JsonInferSchema.scala:134)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1$$anonfun$apply$3.apply(JsonInferSchema.scala:57)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1$$anonfun$apply$3.apply(JsonInferSchema.scala:55)
	at org.apache.spark.util.Utils$.tryWithResource(Utils.scala:2784)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1.apply(JsonInferSchema.scala:55)
	... 25 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2362)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abo

### Recommended Pattern: `badRecordsPath`

Databricks Runtime has [a built-in feature](https://docs.databricks.com/spark/latest/spark-sql/handling-bad-records.html) that saves corrupt records to a given end point. To use this, set the `badRecordsPath`.

This is a preferred design pattern since it persists the corrupt records for later analysis even after the cluster shuts down.

<iframe  
src="//fast.wistia.net/embed/iframe/4t8m5hbwp8?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/4t8m5hbwp8?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

Before getting started, we need some place to store bad records.

We can use our `workingDir` to create a temporary folder for this purpose.

From there we can safely create temp files without any collision from other users.

:NOTE: We defined `workingDir` for you in the call to `Classroom-Setup`

In [0]:
myBadRecords = f"{workingDir}/badRecordsPath"

print(f"""Your temp directory is "{myBadRecords}" """)

Your temp directory is "dbfs:/user/vivek.sivalingam@rhsmith.umd.edu/etl_part_1/etl1_06___corrupt_record_handling_psp/badRecordsPath"

And now let's put your `myBadRecords` to work:

In [0]:
data = """{"a": 1, "b":2, "c":3}|{"a": 1, "b":2, "c":3}|{"a": 1, "b, "c":10}""".split('|')

corruptDF = (spark.read
  .option("badRecordsPath", myBadRecords)
  .json(sc.parallelize(data))
)
display(corruptDF)

a,b,c
1,2,3
1,2,3


-sandbox
See the results in the path specified by `myBadRecords`.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Recall that this directory is backed by S3 and is available to all clusters.

In [0]:
path = "{}/*/*/*".format(myBadRecords)
display(spark.read.json(path))

reason,record
"com.fasterxml.jackson.core.JsonParseException: Unexpected character ('c' (code 99)): was expecting a colon to separate field name and value at [Source: {""a"": 1, ""b, ""c"":10}; line: 1, column: 16]","{""a"": 1, ""b, ""c"":10}"


## Exercise 1: Working with Corrupt Records

### Step 1: Diagnose the Problem

Import the data used in the last lesson, which is located at `/mnt/training/UbiqLog4UCI/14_F/log*`.  Import the corrupt records in a new column `SMSCorrupt`.  <br>

Save only the columns `SMS` and `SMSCorrupt` to the new DataFrame `SMSCorruptDF`.

In [0]:
# TODO
from pyspark.sql.functions import col

SMSCorruptDF = (spark.read
    .option("mode", "PERMISSIVE")
    .option("columnNameOfCorruptRecord","SMSCorrupt")            
    .json("/mnt/training/UbiqLog4UCI/14_F/log*")
    .filter(col('SMSCorrupt')!= 'NULL')
    .select("SMS","SMSCorrupt")
  )

display(SMSCorruptDF)



SMS,SMSCorrupt
null,"{""SMS"":{""Address"":""+98912503####"",""type"":""1"",""date"":""12-24-2013 11:52:14"",""body"":""ANONYMIZED"",""Type"":""1"", ""metadata"": {""name"": ""mr Khojasteh""flash""""}}}"
null,"{""SMS"":{""Address"":""+98912503####"",""type"":""1"",""date"":""12-24-2013 11:52:20"",""body"":""ANONYMIZED"",""Type"":""1"", ""metadata"": {""name"": ""mr Khojasteh""flash""""}}}"
null,"{""SMS"":{""Address"":""+98912503####"",""type"":""1"",""date"":""12-24-2013 11:52:30"",""body"":""ANONYMIZED"",""Type"":""1"", ""metadata"": {""name"": ""mr Khojasteh""flash""""}}}"
null,"{""SMS"":{""Address"":""+98912503####"",""type"":""1"",""date"":""12-24-2013 11:53:26"",""body"":""ANONYMIZED"",""Type"":""1"", ""metadata"": {""name"": ""mr Khojasteh""flash""""}}}"
null,"{""SMS"":{""Address"":""+98912503####"",""type"":""1"",""date"":""12-24-2013 11:53:59"",""body"":""ANONYMIZED"",""Type"":""1"", ""metadata"": {""name"": ""mr Khojasteh""flash""""}}}"
null,"{""SMS"":{""Address"":""+98912503####"",""type"":""1"",""date"":""12-24-2013 11:53:59"",""body"":""ANONYMIZED"",""Type"":""1"", ""metadata"": {""name"": ""mr Khojasteh""flash""""}}}"
null,"{""SMS"":{""Address"":""+98912503####"",""type"":""1"",""date"":""12-24-2013 11:53:26"",""body"":""ANONYMIZED"",""Type"":""1"", ""metadata"": {""name"": ""mr Khojasteh""flash""""}}}"
null,"{""SMS"":{""Address"":""+98912503####"",""type"":""1"",""date"":""12-24-2013 11:52:30"",""body"":""ANONYMIZED"",""Type"":""1"", ""metadata"": {""name"": ""mr Khojasteh""flash""""}}}"


In [0]:
# TEST - Run this cell to test your solution
cols = set(SMSCorruptDF.columns)
SMSCount = SMSCorruptDF.cache().count()

dbTest("ET1-P-06-01-01", True, "SMS" in cols)
dbTest("ET1-P-06-01-02", True, "SMSCorrupt" in cols)
dbTest("ET1-P-06-01-03", 8, SMSCount)

print("Tests passed!")

Tests passed!

-sandbox
Examine the corrupt records to determine what the problem is with the bad records.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** Take a look at the name in metadata.

The entry `{"name": "mr Khojasteh"flash""}` should have single quotes around `flash` since the double quotes are interpreted as the end of the value.  It should read `{"name": "mr Khojasteh'flash'"}` instead.

The optimal solution is to fix the initial producer of the data to correct the problem at its source.  In the meantime, you could write ad hoc logic to turn this into a readable field.

### Step 2: Use `badRecordsPath`

Use the `badRecordsPath` option to save corrupt records to the directory specified by the `corruptPath` variable below.

In [0]:
# TODO

corruptPath = f"{workingDir}/corruptSMS"

SMSCorruptDF2 = (spark.read
  .option("badRecordsPath", corruptPath)
  .json("/mnt/training/UbiqLog4UCI/14_F/log*")
)
display(SMSCorruptDF2)

Application,Bluetooth,Call,Location,SMS,WiFi
null,null,null,null,null,"List(74:ea:3a:a2:e1:b4, Saba, [WPA-PSK-TKIP], 2437, -88, Thursday, January 9, 2014 11:56:44 PM Iran Standard Time)"
null,null,null,null,null,"List(90:94:e4:83:57:83, Milad, [WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP][WPS], 2472, -85, Thursday, January 9, 2014 11:56:44 PM Iran Standard Time)"
null,null,null,null,null,"List(74:ea:3a:a2:e1:b4, Saba, [WPA-PSK-TKIP], 2437, -88, Thursday, January 9, 2014 11:57:19 PM Iran Standard Time)"
null,null,null,null,null,"List(90:94:e4:83:57:83, Milad, [WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP][WPS], 2472, -86, Thursday, January 9, 2014 11:57:19 PM Iran Standard Time)"
null,null,null,null,null,"List(74:ea:3a:a2:e1:b4, Saba, [WPA-PSK-TKIP], 2437, -88, Thursday, January 9, 2014 11:57:19 PM Iran Standard Time)"
null,null,null,null,null,"List(90:94:e4:83:57:83, Milad, [WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP][WPS], 2472, -86, Thursday, January 9, 2014 11:57:19 PM Iran Standard Time)"
null,null,null,null,null,"List(74:ea:3a:a2:e1:b4, Saba, [WPA-PSK-TKIP], 2437, -88, Thursday, January 9, 2014 11:57:44 PM Iran Standard Time)"
null,null,null,null,null,"List(90:94:e4:83:57:83, Milad, [WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP][WPS], 2472, -85, Thursday, January 9, 2014 11:57:44 PM Iran Standard Time)"
null,null,null,null,null,"List(74:ea:3a:a2:e1:b4, Saba, [WPA-PSK-TKIP], 2437, -88, Thursday, January 9, 2014 11:58:19 PM Iran Standard Time)"
null,null,null,null,null,"List(90:94:e4:83:57:83, Milad, [WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP][WPS], 2472, -87, Thursday, January 9, 2014 11:58:19 PM Iran Standard Time)"


In [0]:
# TEST - Run this cell to test your solution
SMSCorruptDF2.count()

testPath = f"{workingDir}/corruptSMS/*/*/*"
corruptCount = spark.read.json(testPath).count()

dbTest("ET1-P-06-02-01", True, corruptCount >= 8)

print("Tests passed!")

Tests passed!

## Review
**Question:** By default, how are corrupt records dealt with using `spark.read.json()`?  
**Answer:** They appear in a column called `_corrupt_record`.

**Question:** How can a query persist corrupt records in separate destination?  
**Answer:** The Databricks feature `badRecordsPath` allows a query to save corrupt records to a given end point for the pipeline engineer to investigate corruption issues.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Cleanup<br>

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [0]:
%run "./Includes/Classroom-Cleanup"

Removing: dbfs:/user/vivek.sivalingam@rhsmith.umd.edu/etl_part_1/etl1_06___corrupt_record_handling_psp/badRecordsPath/
Removing: dbfs:/user/vivek.sivalingam@rhsmith.umd.edu/etl_part_1/etl1_06___corrupt_record_handling_psp/corruptSMS/
Removing: dbfs:/user/vivek.sivalingam@rhsmith.umd.edu/etl_part_1/etl1_06___corrupt_record_handling_psp

,Send


## Next Steps

Start the next lesson, [Loading Data and Productionalizing]($./07-Loading-Data-and-Productionalizing).

## Additional Topics & Resources

**Q:** Where can I get more information on dealing with corrupt records?  
**A:** Check out the Spark Summit talk on <a href="https://databricks.com/session/exceptions-are-the-norm-dealing-with-bad-actors-in-etl" target="_blank">Exceptions are the Norm: Dealing with Bad Actors in ETL</a>

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>